In [18]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from webencodings import labels

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

input_text = "Transformer are the"
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"].to(device)
iters = []
n_step3 = 8
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_step3):
        iter = dict()
        iter["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # store tokens with highest probs
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            iter[f"Choice {choice_idx+1}"] = token_choice
            
            # append predicted next token to input
            input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iters.append(iter)

df = pd.DataFrame(iters)
df.to_html("/Users/admin/Developer/PyProjects/DeepLearing/output/output.html")

#use generate
output = model.generate(input_ids, do_sample=False, max_new_tokens=n_step3)
print(tokenizer.decode(output[0]))
print("-"*40)
#reproduce unicorn stories
max_length = 128
input_text = """In a shocking finding, scientist discovered \
a herd of unicorns living in a remote, previously unexplored \
valley, in the Andes Mountains. Even more surprising to the \
researchers was the fact that the unicorns spoke perfect English.\n\n
"""
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformer are the most most most most most powerful powerful powerful powerful powerful weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon
----------------------------------------
Transformer are the most most most most most powerful powerful powerful powerful powerful weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon we

## Greedy Search vs Beam Search Decoding vs Stochastic Beam Search
### Greedy Search （贪心搜索）
* 每一步都选择概率最高的一个单词作为下一个token，只跟踪一条最优路径，不考虑其他选项。
* 公式：$ Y_{t+1} = \arg \max_{ w}P(w \mid Y_{1:t}) $ <br><br>
* 计算快实现简单，但容易卡在局部最优，因为他不考虑未来选项，且缺乏多样性，相同输入总是相同输出。<br>

### Beam Search Decoding（集束搜索）
* 维护 k 条可能的路径（beam size = k），在每一步选择 k 个最可能的序列继续扩展，而不是只选一个最优路径。
* 最终返回概率最高的完整句子。
* 公式：近似最大化序列概率 $ Y^* = \arg \max_{ y}P(Y \mid X) $
1. 设 beam_size = k（比如 k=3）
2. 在每一步，计算所有扩展路径的概率，保留 k 个最优的序列。
3. 直到句子结束符 <EOS>，选择最高概率的完整序列。
<br>
比贪心搜索更优，避免局部最优解。可调控 k 值，在质量和计算成本之间取得平衡。但 计算开销较大，k 越大，计算量越高；仍然可能丢失全局最优解，如果 k 过小，搜索空间不够大。

### Stochastic Beam Search（随机集束搜索）

* 传统 Beam Search 只保留最高概率的路径，而 Stochastic Beam Search 会随机选择 k 个候选路径。
* 引入随机性，避免所有生成的文本都完全相同，提升文本多样性
* 过程
1. 采样候选路径，而不是每次都选 top-k 的最优路径。
2. 在 beam_size 内，使用温度系数调整概率：
$ P(w) \propto P(w)^{1/T} $ <br>
    T > 1 → 选择更随机的词 <br>
    T < 1 → 选择更确定的词 <br>

生成更多样化的文本，适用于对话、故事生成等任务。不会死板地选取最优解，更像人类写作风格。<br>
质量不一定最优，可能选到概率较低的词。需要调节超参数，如 T 和 k，以保证质量。<br>

In [28]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    """
    single token log probability from logits
    :param logits: 
    :param labels: 
    :return: 
    """
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sequence_logprobs(model, labels, input_len = 0):
    """
    sequence log probability
    :param model: 
    :param labels: 
    :param input_len: 
    :return: 
    """
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], labels[:, 1:])
        seq_log_probs = torch.sum(log_probs[:, input_len: ])
    return seq_log_probs

logp = sequence_logprobs(model, output_greedy, input_len=len(input_ids[0]))
print("-/"*20)
print(tokenizer.decode(output_greedy[0]))
print(f"Sequence log probs: {logp:.2f}")

print("-bs-"*20)
print('beam search:\n')
# beam search
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
 do_sample=False)
logp = sequence_logprobs(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

# beam search with ngram
print("-bs ng-"*20)
print("\n")
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5,
 do_sample=False, no_repeat_ngram_size=3)
logp = sequence_logprobs(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/
Transformer are the most most most most most powerful powerful powerful powerful powerful weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon
Sequence log probs: -2.31
-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/-/
beam

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformer are the most most most most most powerful powerful powerful powerful powerful weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon weapon

log-prob: -2.31
-/ngram-/ngram-/ngram-/ngram-/ngram-/ngram-/ngram-/ngram-/ngram-/ngram-/ngram-/ngram-/ngram-/ng

> <strong> N-gram Penalty（N-gram 惩罚）</strong>  : N-gram Penalty（N-gram 惩罚）是一种 解码策略，用于 防止重复生成相同的 N-gram 片段，尤其在 文本生成任务（如机器翻译、文本摘要、对话系统）中，能有效减少 重复问题。<br>
> N-gram 指的是由 N 个连续 token 组成的短语
> 重复模式 发生的原因: <br>
>   a. 语言模型学习到了某些短语的高概率，导致它 倾向于重复生成。 <br>
>   b. 在 Beam Search 或 Greedy Search 过程中，较高概率的 token 会不断被选中，形成重复。<br>
> How it works? <br>
>   1. 记录 已经生成的 N-gram 片段 <br>
>   2. 当解码过程中尝试生成下一个 token 时: 如果这个 token 形成了已存在的 N-gram，则 禁止生成（分数设为极低）。这样就可以 避免完全相同的重复短语。<br>

## Sampling Methods

Reduces repetitions.

### 随机采样
最简单的采样方法是在每个时间步随机从模型输出的整个词汇表的概率分布中进行采样。 <br>

$P(y_t = w_i | y_{<t}, \mathbf{x}) = \text{softmax}(z_{t,i}) = \frac{\exp(z_{t,i})}{\sum_{j=1}^{|V|} \exp(z_{t,j})} $ <br>

V 表示词表基数，可以通过增加一个温度参数 T在softmax操作之前缩放logits，以此控制输出结果的多样性。<br>

$P(y_t = w_i | y_{<t}, \mathbf{x}) = \frac{\exp(z_{t,i} / T)}{\sum_{j=1}^{|V|} \exp(z_{t,j} / T)}$ <br><br>

调整T来控制概率分布的形状。T<= 1, 分布在原点附近变得尖锐，稀有的token被抑制；当温度 T≫1 时，分布变得平坦，每个token变得同样可能。 <br><br>
<img src="./imgs/temperature_contrl_logits_scale.png" width="400"/> <br>

* T 越小，概率分布越陡峭，输出更确定（接近贪心解）。
* T 越大，概率分布越平缓，采样更随机（提升多样性但可能导致错误）。
* 通常T 设为 0.7 ~ 1.0 之间，以平衡确定性与多样性
